In [ ]:
import os
import cv2
import random
import time
import pygame
import numpy as np
from collections import deque
from threading import Thread
from tkinter import *
from tensorflow.keras.models import load_model
import mediapipe as mp
pygame.mixer.init()
song_folder = r"C:\Users\KIIT\Desktop\project\songs"
current_song = None
detected_mood = "Neutral"
music_playing = False
music_paused = False
exit_flag = False
current_volume = 0.5
gesture_detection_enabled = True


emotion_model = load_model("facial_expression_model.h5") 
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

root = Tk()
root.title("Mood-Based Music Player")
root.geometry("700x600")
root.configure(bg='white')


mood_var = StringVar()
song_var = StringVar()
status_var = StringVar()

# Labels
Label(root, text="Detected Mood:", font=("Helvetica", 16)).pack(pady=5)
mood_label = Label(root, textvariable=mood_var, font=("Helvetica", 20), bg="gray", fg="white", width=20)
mood_label.pack(pady=10)

Label(root, text="Song Status:", font=("Helvetica", 14)).pack(pady=5)
Label(root, textvariable=song_var, font=("Helvetica", 14), bg="lightgray", width=50).pack(pady=10)

Label(root, textvariable=status_var, font=("Helvetica", 12), fg="green").pack(pady=5)


def update_mood_color(color):
    mood_label.config(bg=color)

def detect_emotion(frame):
    """Use CNN model for emotion detection"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        return "Neutral"

    (x, y, w, h) = faces[0]
    face_roi = gray[y:y+h, x:x+w]
    resized = cv2.resize(face_roi, (48, 48))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 48, 48, 1))

    predictions = emotion_model.predict(reshaped, verbose=0)
    return emotion_labels[np.argmax(predictions)]

def detect_mood():
    global detected_mood
    cap = cv2.VideoCapture(0)
    status_var.set("Detecting Mood...")
    mood_var.set("Detecting...")
    update_mood_color("gray")

    start_time = time.time()
    mood_counts = {}

    while time.time() - start_time < 7:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        current_mood = detect_emotion(frame)
        mood_counts[current_mood] = mood_counts.get(current_mood, 0) + 1

        # Display on frame
        cv2.putText(frame, f"Mood: {current_mood}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Mood Detector", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if mood_counts:
        detected_mood = max(mood_counts, key=mood_counts.get)
        mood_var.set(detected_mood)
        update_mood_color("lightblue")
        status_var.set(f"Mood detected: {detected_mood}")
    else:
        detected_mood = "Neutral"
        mood_var.set("Neutral")
        update_mood_color("gray")
        status_var.set("Mood detection failed")

def play_song(mood):
    global current_song, music_playing, music_paused

    songs = [f for f in os.listdir(song_folder) if mood.lower() in f.lower()]
    if not songs:
        songs = os.listdir(song_folder)

    if not songs:
        song_var.set("No songs found.")
        return

    current_song = random.choice(songs)
    pygame.mixer.music.load(os.path.join(song_folder, current_song))
    pygame.mixer.music.set_volume(current_volume)
    pygame.mixer.music.play()
    music_playing = True
    music_paused = False
    song_var.set(f"Playing: {current_song}")

def stop_song():
    global music_playing, music_paused
    pygame.mixer.music.stop()
    music_playing = False
    music_paused = False
    song_var.set("Music stopped")
    status_var.set("")

def start_detection_and_play_song():
    detect_mood()
    play_song(detected_mood)

Button(root, text="Detect Mood & Play Song", font=("Helvetica", 14), command=lambda: Thread(target=start_detection_and_play_song).start(), bg="#4CAF50", fg="white").pack(pady=15)
Button(root, text="Stop", font=("Helvetica", 14), command=stop_song, bg="#f44336", fg="white").pack()

if not os.path.exists(song_folder):
    os.makedirs(song_folder)

root.mainloop()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
